<a href="https://colab.research.google.com/github/victordot/Inteligencia-Artificial-2025-2/blob/main/03%20-%20modelo_con_preprocesado_Target_Encoding_y_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"victordocampot","key":"0f8ade079b2df30ff64919a0ea693ea6"}'}

In [ ]:
import os
import shutil

# Crear carpeta oculta .kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

source_path = "kaggle.json"
destination_path = "/root/.kaggle/kaggle.json"

if os.path.exists(destination_path):
    os.remove(destination_path)

shutil.move(source_path, destination_path)


'/root/.kaggle/kaggle.json'

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!ls -lh

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.21GB/s]
total 30M
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data
-rw-r--r-- 1 root root  30M Sep 16 01:46 udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip


In [ ]:
!pip install category_encoders xgboost scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import zipfile

with zipfile.ZipFile("udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
    zip_ref.extractall("dataset_saber_pro")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from category_encoders import TargetEncoder
import xgboost as xgb

# Cargar datos
df = pd.read_csv('/content/dataset_saber_pro/train.csv')

print(f"Dimensiones del dataset: {df.shape}")
df.head()

Dimensiones del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
def preprocess_manual_features(df_input):
    df = df_input.copy()

    # 1. Eliminar columnas innecesarias o duplicadas
    cols_to_drop = ['ID', 'F_TIENEINTERNET.1']
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

    # 2. Mapeo de Educación (Padre y Madre)
    # Asignamos un valor numérico mayor a mayor nivel educativo
    edu_map = {
        'Ninguno': 0, 'No sabe': 0, 'No Aplica': 0,
        'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
        'Postgrado': 9
    }
    df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(edu_map).fillna(-1) # -1 para nulos nuevos
    df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(edu_map).fillna(-1)

    # 3. Mapeo de Estrato
    estrato_map = {
        'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2,
        'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6
    }
    df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(estrato_map).fillna(2)

    # 4. Mapeo de Horas de Trabajo
    trabajo_map = {
        '0': 0, 'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2, 'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4
    }
    df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].map(trabajo_map).fillna(0)

    # 5. Mapeo de Valor Matrícula
    matricula_map = {
        'No pagó matrícula': 0, 'Menos de 500 mil': 1,
        'Entre 500 mil y menos de 1 millón': 2,
        'Entre 1 millón y menos de 2.5 millones': 3,
        'Entre 2.5 millones y menos de 4 millones': 4,
        'Entre 4 millones y menos de 5.5 millones': 5,
        'Entre 5.5 millones y menos de 7 millones': 6,
        'Más de 7 millones': 7
    }
    df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(matricula_map).fillna(2) # Imputar mediana aprox

    return df

# Aplicamos la limpieza manual
df_clean = preprocess_manual_features(df)

# 6. Codificación del Target
if 'RENDIMIENTO_GLOBAL' in df_clean.columns:
    target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
    df_clean['RENDIMIENTO_GLOBAL'] = df_clean['RENDIMIENTO_GLOBAL'].map(target_map)

    # Separamos X e y
    X = df_clean.drop(columns=['RENDIMIENTO_GLOBAL'])
    y = df_clean['RENDIMIENTO_GLOBAL']
else:
    # Caso para el archivo de test.csv (sin target)
    X = df_clean
    y = None

print("Datos preprocesados manualmente. Listos para el Pipeline.")

Datos preprocesados manualmente. Listos para el Pipeline.


In [ ]:
# Numéricas
numeric_features = [
    'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE', 'F_ESTRATOVIVIENDA',
    'E_HORASSEMANATRABAJA', 'E_VALORMATRICULAUNIVERSIDAD',
    'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'
]

# Categóricas Binarias/Nominales  OneHot
categorical_features = [
    'F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
    'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR'
]

# Categóricas de Alta Cardinalidad Target Encoding
high_cardinality_features = [
    'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO'
]


# 1. Para Numéricas: Imputar mediana y Escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. Para Categóricas Simples: Imputar 'Missing' y One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 3. Para Alta Cardinalidad: Target Encoding
high_cardinality_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('target_enc', TargetEncoder(smoothing=10))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('high_card', high_cardinality_transformer, high_cardinality_features)
    ],
    remainder='drop'
)



Pipeline de preprocesamiento construido.


In [ ]:
# Dividir en Train y Validación para probar
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el Modelo Completo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=4,
        n_estimators=200,    # Número de árboles
        learning_rate=0.1,   # Velocidad de aprendizaje
        max_depth=6,         # Profundidad de los árboles
        random_state=42,
        n_jobs=-1
    ))
])

# Entrenar
print("Entrenando modelo... esto puede tardar unos minutos.")
model.fit(X_train, y_train)

# Predecir
y_pred = model.predict(X_val)

# Evaluar
acc = accuracy_score(y_val, y_pred)
print(f"\n>>> Accuracy en Validación: {acc:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred))

Entrenando modelo... esto puede tardar unos minutos.

>>> Accuracy en Validación: 0.4325

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.46      0.55      0.50     34597
           1       0.33      0.27      0.29     34455
           2       0.33      0.27      0.30     34324
           3       0.55      0.63      0.59     35124

    accuracy                           0.43    138500
   macro avg       0.42      0.43      0.42    138500
weighted avg       0.42      0.43      0.42    138500



In [ ]:
import pandas as pd
import numpy as np

reverse_map = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

try:
    df_test = pd.read_csv('/content/dataset_saber_pro/test.csv')
except FileNotFoundError:
    print("Error: Asegúrate de que 'test.csv' esté cargado en tu entorno.")


# Guardar los IDs originales para el envío
test_ids = df_test['ID']

# Aplicar el preprocesamiento manual (ordinales, eliminación de duplicados)
df_test_clean = preprocess_manual_features(df_test)

#Predicciones modelos
print("Generando predicciones en el conjunto de prueba...")
predictions_num = model.predict(df_test_clean)

#Formatear para envio

# Convertir las predicciones numéricas a las etiquetas de texto requeridas
predictions_label = [reverse_map[pred] for pred in predictions_num]

# Crear el DataFrame de envío
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predictions_label
})

# Guardar el archivo CSV
submission.to_csv('submission_xgb.csv', index=False)


Generando predicciones en el conjunto de prueba...

🎉 Archivo 'submission_xgb.csv' generado exitosamente.
Listo para subir a Kaggle.
